# Graph Neural Network
---
> Graph neural network model for vertex dynamics and tension prediction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from os import path
import datetime

from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T

from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx
from simgnn.datautils import load_array, load_graph

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,10) # use larger for presentation
matplotlib.rcParams['font.size']= 16 # use 14 for presentation

In [3]:
from simgnn.datasets import persistence_loss, VertexDynamics, HaraMovies, HaraAblation
from simgnn.transforms import Pos2Vec, ScaleVelocity, ScaleTension, ScalePressure, Reshape_x, TransformTension, RecoilAsTension
# from torch_geometric.utils import to_undirected as T_undir

In [4]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
dtype = torch.float32
print(f'Defaults:\n |-device: {device}\n |-dtype : {dtype}')

Defaults:
 |-device: cuda
 |-dtype : torch.float32


## Data, Stat-s, Transforms

Notes for normalisation param-s:

- Use `simgnn.transforms` to normalise real--Hara movies and ablation,  and simulated movies. Use same normalisation constants for all simulated movies.
- For ablation movies, convert to  HaraMovies length scale (pixels) and use same `l_av` as for HaraMovies dataset.

---
|Dataset| Avg. edge len.|Tension/Recoil| Transforms (Normalisation)|Notes|
|:---:|---:|---:|:---|:---:|
|`single_distr_sims` | 1.08 a.u. | range=[0.00131, 4.26]; sd=0.637; median=0.305; mean=0.578; | `[Pos2Vec(scale=10*1.0)`, `ScaleVelocity(0.5*1.0)`, `ScaleTension(0.634,shift=0.6)]`| synthetic; w/o base tension |
|`unimodal_wbasetens_sims` | 0.906 a.u. | range=[0.407, 5.47]; sd=0.729; median=1.48; mean=1.6| `[Pos2Vec(scale=10*1.0)`, `ScaleVelocity(0.5*1.0)`, `ScaleTension(0.634,shift=0.6)]`| synthetic; w/ non-zero base tension |
|`HaraMovies`  | 26.32 pixels||`[Pos2Vec(scale=10*26.32)`, `ScaleVelocity(0.5*26.32)]`| tissue movies w/o force data|
|`HaraAblation`|$26.32\cdot\frac{0.4}{0.21}$ pixels| range=[0.0239, 2.61]; sd=0.651; median=0.673; mean=0.826 | `[Pos2Vec(scale=10*50.13), ScaleVelocity(0.5*50.13)]`| short movies w/ recoil data; frame rate:dataset contains vertex positions only from every 10th frame |

- Logarithm of tension: ln(Tension/Recoil {raw values})
```
Single_distr_sims:        range: [-6.63 1.45]; s.d.: 1.22 || median: -1.19; mean: -1.2;
Unimodal_wbasetens_sims : range: [-0.9  1.7]; s.d.: 0.451 || median: 0.39; mean: 0.371;
Hara ablation:            range: [-3.73 0.958]; s.d.:0.962 || median:-0.396; mean:-0.547;
```


---
- **Normalisation param-s**

In [5]:
# Data normalisation param-s : sim="default" for simulation data
# Average edge lengths
l_0 = {'sim':1.0, 'single_distr_sims':1.1, 'unimodal_wbasetens_sims':0.91, 'hara':26.32, 'abln':50.13}

# Stat-s for raw tension/recoil values
t_av = {'sim':0.6, 'single_distr_sims': 0.58, 'unimodal_wbasetens_sims': 1.6, 'abln': 0.83}
t_sd = {'sim':0.63, 'single_distr_sims': 0.64, 'unimodal_wbasetens_sims':0.73 , 'abln': 0.65}

### Datasets

In [6]:
!rm -dr simgnn_data/single_distr_sims/train/processed
!rm -dr simgnn_data/single_distr_sims/val/processed
!rm -dr simgnn_data/unimodal_wbasetens_sims/train/processed
!rm -dr simgnn_data/unimodal_wbasetens_sims/val/processed

!rm -dr simgnn_data/hara_movies/processed/
!rm -dr simgnn_data/hara_ablation/processed/

# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg001/processed/
# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg003/processed/
# !rm -dr simgnn_data/hara_movies_as_sep_datasets/hara_seg005/processed/

**Input features and normalisation param-s (transforms)**

In [7]:
# Input features
window_size = 5
print(f'> Set features window size (#past velocities) to {window_size}')

# Normalisation
# for simulated data (simul params ~ normal distr-s): l0_sim=1.0, l0_HaraMovie: l0=26.32
Tnrm = {} # dict of transforms

# Default for all simulations:
Tnrm['sim'] = T.Compose([Pos2Vec(scale=10*l_0['sim']) , ScaleVelocity(0.5*l_0['single_distr_sims']),
                         ScaleTension(t_sd['sim'], shift=t_av['sim']),
                         Reshape_x((-1,window_size*2)) ] )

# w/o base contractility:
Tnrm['single_distr_sims'] = T.Compose([Pos2Vec(scale=10*l_0['single_distr_sims']) , ScaleVelocity(0.5*l_0['single_distr_sims']),
                                       ScaleTension(t_sd['single_distr_sims'], shift=t_av['single_distr_sims']),
                                       Reshape_x((-1,window_size*2)) ] )

# w/ base contractility:
Tnrm['unimodal_wbasetens_sims'] = T.Compose([Pos2Vec(scale=10*l_0['unimodal_wbasetens_sims']) , ScaleVelocity(0.5*l_0['unimodal_wbasetens_sims']),
                                             ScaleTension(t_sd['unimodal_wbasetens_sims'], shift=t_av['unimodal_wbasetens_sims']),
                                             Reshape_x((-1,window_size*2)) ] )

# Hara ablation norm
Tnrm['abln'] = T.Compose([Pos2Vec(scale=10*l_0['abln']), ScaleVelocity(0.5*l_0['abln']), RecoilAsTension(),
                          ScaleTension(t_sd['abln'], shift=t_av['abln']),
                          Reshape_x((-1,window_size*2))]) 

# Hara movie dataset norm
Tnrm['hara'] = T.Compose([Pos2Vec(scale=10*l_0['hara']) , ScaleVelocity(0.5*l_0['hara']), Reshape_x((-1,window_size*2))]) 

> Set features window size (#past velocities) to 5


**Load datasets**

In [9]:
hara_abln = HaraAblation('simgnn_data/hara_ablation/', window_size=window_size, transform=Tnrm['abln'], smoothing=True, sma_lag_time=3)
hara = HaraMovies('simgnn_data/hara_movies/',window_size=window_size, transform=Tnrm['hara'], smoothing=True, sma_lag_time=4)

# Hara movie datasets
# seg001 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg001/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)
# seg001_raw = HaraMovies('simgnn_data/hara_movies/', window_size=window_size, transform=Tnorm_hara)
# seg003 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg003/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)
# seg005 = HaraMovies('simgnn_data/hara_movies_as_sep_datasets/hara_seg005/', window_size=window_size, transform=Tnorm_hara,smoothing=True, sma_lag_time=4)

# Simulation datasets
# rand base contractility
sim1 = VertexDynamics('./simgnn_data/unimodal_wbasetens_sims/train/', window_size=window_size,
                      transform=Tnrm['unimodal_wbasetens_sims']
                     )
sim1_val = VertexDynamics('./simgnn_data/unimodal_wbasetens_sims/val/', window_size=window_size,
                          transform=Tnrm['unimodal_wbasetens_sims']
                         )

# w/o base contractility
sim2 = VertexDynamics('./simgnn_data/single_distr_sims/train/', window_size=window_size,
                      transform=Tnrm['single_distr_sims'])
sim2_val = VertexDynamics('./simgnn_data/single_distr_sims/val/', window_size=window_size,
                          transform=Tnrm['single_distr_sims']
                         )


datasets_dict = {'train': sim1,
                 'val': sim1_val,
                 'val2': sim2_val,
                 'hara' : hara,
                 'abln': hara_abln}
dataset_legend={'train': 'Train(sim1)',
                'val': 'Val(sim1)',
                'val2': 'Val(sim2)',
                'hara': 'Hara',
                'abln': 'Recoil'}

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


In [10]:
# k='val2'
# print('{}\n{} : {}\nsize:{}\n\nFrame 1:\n{}'.format(dataset_legend[k],k,datasets_dict[k],len(datasets_dict[k]),datasets_dict[k][0]))
# plt.figure(figsize=[2.5,2.5])
# nx.draw(to_networkx(datasets_dict[k][0]),pos=dict(enumerate(datasets_dict[k][0].pos.numpy())), node_size=10)

**Persistence Losses**

In [11]:
# print('Persistence:')
# for k in datasets_dict:
#     if k=='abln':print('Ignoring ablation data velocity');continue
#     print(f'\t- {dataset_legend[k]}: {persistence_loss(datasets_dict[k])}')
    
# print(len(datasets_dict['train']))

**To-Do**👷🚧
- need a func-n w/ a **rollout error**,
- convert vel-y error to **position error**, e.g. "speed"+"direction"(angle/dot product etc.)
- *Training loop*:
    - [ ] Combine `Message` and `AggregateUpdate` into a graph layer `GraphBlock`
- [ ] Write *prediction stage*: read \{test, val, train\} data for rollout error measurements.
- [ ] Experiments (**save all models** w/ backups; will need to test on the real tensions)<br>
*Params*: edge attrib-s, cell layer, arch(skip con-s), input noise (for long term prediction). *Errors*: 1-step, and rollout error (check after training at test time).
    1. Experiment with GN arch-s w/ *residual* (w/ skip) and *non-residual* architectures.
    1. *Edge directions experiment*: train w/ and w/o edge dir-s, do edge dir-s help to speed up training?
    1. *Cell layer*: does having dedicated cell processing layer help to increase the accuracy?
    1. For best (resnet or non-resnet) run *number of layers and dim-n sizes*. Try deep nets, how does accuracy change with increasing the depth of the net?
    1. If resnet is better: try w/and w/o edge dir-s and "cell layers".
    1. Number of previous velocities (window size).
    1. Rollout experiment 1— *input noise*: according to Sanchez-Gonzalez, *et al.* \[ASG2020\], Brownian noise improves rollout accuracies (long term accuracy of the whole movie, and/or prediction stability/robustness).
    1. Rollout experiment 2— *rollout training*: train directly on rollout, i.e. use 5- and 10-step loss instead of a 1-step loss for training (slower training).
    1. Optional:
        - compare MLP vs CONV layers for message passing.
        - try with dynamic graphs (construct graphs on the fly based on relative positions, and use cell edges and cell attrib only for queries on `Y_edge`, `Y_cell`).
- [ ] Ablation dataset (*real*).

---
**DOING**🛠
1. Construct graph net without skip connections.
1. Combine Message and AggregateUpdate into a graph layer GraphNet (GN) block, a more general block, that can be composed into a deep residual network. "AddGN" block, w/ `AddGN(x) = f(x)+x` form (in fact, where it's possible make all blocks with this form.
1. Construct residual net out of GNs.
---

- **Examples**:
    - General "Message Passing" schemes: a nice example for composite graph layer –"meta layer" consisting of "edge", "node" and "global" layers [link](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.meta.MetaLayer)

**Node-to-Cell Encoding/Pooling Layer**:
1. Initiate node-to-cell edge attr-s as (source) node attr-s `x[node2cell_index[0]]`.
1. Compute node-to-cell edge attr-s using MLP: `e_n2c = MLP( x[node2cell_index[0]] )`
1. Aggregate node-to-cell edge attr-s as cell attr-s : `x_cell = Aggregate(e_n2c)`
1. Compute new cell attr-s using (encodes `x_cell` into cell attr-s) : `h_cell = MLP_Cell_encoder( x_cell )`

```python
n2c_model = mlp(...) # "message", just node-wise MLP
cell_aggr = Aggregate()
cell_enc = mlp(...)

e_n2c = n2c_model(data.x)[data.node2cell_index[0]]
x_cell = cell_aggr(data.cell_pressures.size(0), data.node2cell_index, e_n2c)
h_cell = cell_enc(x_cell)
```

---
- **Adding noise from M steps**: Sum of M normal rand. var-s results in normal var. w/ variance M and s.t.d.=sqrt(M):
```python
x = np.random.normal(size=(5,1000))
y = x.sum(axis=0)
z = np.random.normal(size=(1,1000))*np.sqrt(5)
plt.hist(x.ravel(),bins=50,label='x',density=True)
plt.hist(y        ,bins=50,label='y',density=True)
plt.hist(z.ravel(),bins=50,label='z',density=True,alpha=.5)
plt.legend();
```

---

## Tension Models

In [ ]:
from simgnn.train import train_model, write_log, load_log, plot_losses
from simgnn.train import predict, predict_batch, plot_velocity_predictions, plot_tension_prediction

In [36]:
from simgnn.nn import mlp

Need
- Enc
- Dec
- Block


In [37]:
class IndependentEncoder (torch.nn.Module):
    '''
    Layer for encoding node and edge features using independent MLPs.
    
    The last layer is always a plain linear layer, i.e. it has no dropout/activation.
    '''
    def __init__(self, in_dims, latent_dim, hidden_dims=[],mlp_kwargs={}):
        '''
        Arg-s:
            - in_dims : number of input dimensions. A dict of integers w/ keys "node" and "edge".
            - latent_dim : number of output dimensions for encoder MLPs.
            - hidden_dims : a list of hidden dimensions.
        
        Note:
            - The same `latent_dim`, `hidden_dims`, and `mlp_kwargs` arg-s are used for both edge and node MLPs.
        '''
        super(IndependentEncoder, self).__init__()
        self.node_mlp = mlp(in_dims['node'],latent_dim, hidden_dims=hidden_dims,**mlp_kwargs)
        self.edge_mlp = mlp(in_dims['edge'],latent_dim, hidden_dims=hidden_dims,**mlp_kwargs)
        
    def forward(self, x, e):
        return self.node_mlp(x), self.edge_mlp(e)

In [38]:
hara_abln[0]

CellData(cell2node_index=[2, 38], edge_attr=[29, 2], edge_index=[2, 29], edge_recoils=[29], edge_tensions=[29], node2cell_index=[2, 38], pos=[24, 2], x=[24, 10])

In [42]:
Enc = IndependentEncoder({'node':10,'edge':2}, 16, hidden_dims=[32])

In [44]:
x_e, e_e = Enc(hara_abln[0].x,hara_abln[0].edge_attr)

In [ ]:
class SingleMP_Tension(torch.nn.Module):
    '''
    Returns tuple (y_pred, None, None):
    - y_pred: is a node-wise output (e.g. node velocity)
    - The two `None`s are just place holders to make model compatible with training function in `train.py`.
    '''
    def __init__(self, node_in_features=10, node_out_features=2, edge_in_features=2,
                 message_out_features=5, message_hidden_dims=[10], update_hidden_dims = [],aggr='mean',
                 tension_out_features=1, tension_hidden_dims=[5], **mlp_kwargs):
        '''
        Arg-s:
        - node_in_features : #input node features
        - node_out_features: #output node features
        - edge_in_features : #input edge features
        - message_out_features : #message features (edge-wise messages, can be considered as new or intermediate edge features )
        - message_hidden_dims : list of #dims for message MLP=phi. For edge s->t: m_st = phi([x_t - x_s, e_st]).
        - update_hidden_dims : list of #dims for update MLP=gamma. For node i : x_i' = gamma(x_i, Aggregate(m_si))
        - Optional kwargs for both MLPs: defaults are `dropout_p = 0`, `Fn = ReLU`, `Fn_kwargs = {}`.
        '''
        super(SingleMP_Tension, self).__init__()

        self.message = DiffMessage(node_in_features+edge_in_features,
                                   message_out_features,
                                   hidden_dims=message_hidden_dims, **mlp_kwargs)
        self.relu = torch.nn.ReLU()
        self.aggr_update = AggregateUpdate(node_in_features+message_out_features,
                                           node_out_features, hidden_dims=update_hidden_dims, aggr=aggr, **mlp_kwargs)
        self.tension_mlp = mlp(message_out_features, tension_out_features, 
                               hidden_dims=tension_hidden_dims, **mlp_kwargs)

    def forward(self, data):
        # convert to undirected graph : cat([e_ij, e_ji])
        edge_index = torch.cat([ data.edge_index, torch.stack([data.edge_index[1],
                                                               data.edge_index[0]], dim=0) ], dim=1).contiguous()
        # edge features for undirected graph : e_ij = - e_ji
        edge_attr  = torch.cat([ data.edge_attr, -data.edge_attr], dim=0).contiguous()

        # message
        src, tgt = data.x[edge_index[0]], data.x[edge_index[1]] # src, tgt features
        m_ij = self.relu( self.message(src, tgt, edge_attr) )

        # aggregate and update stages
        x_out = self.aggr_update( data.x, edge_index, m_ij) # leave last layer as linear, i.e. no ReLU()
        
        # tension model
        e_out = self.tension_mlp(m_ij[:m_ij.size(0)//2,:] +  m_ij[m_ij.size(0)//2 :,:])
        
        return x_out, e_out.reshape((e_out.size(0),)), None